In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import xgboost as xgb
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from sklearn.preprocessing import MinMaxScaler



In [ ]:
# Load dataset
file_path = "Emissions_Totals_E_All_Data.csv"
df = pd.read_csv(file_path)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ruptures as rpt
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import periodogram
import numpy as np

def detect_seasonality_period(data: pd.Series) -> int:
    """
    Detect dominant seasonality period in a time series using periodogram.
    If no clear peak is found, return 1 (no seasonality).
    """
    freqs, psd = periodogram(data.dropna())
    if len(freqs) == 0 or len(psd) == 0:
        return 1
    
    dominant_freq = freqs[np.argmax(psd[1:]) + 1]  # Ignore the first zero frequency
    if dominant_freq == 0:
        return 1
    period = int(round(1 / dominant_freq))
    return max(period, 1)

def analyze_timeseries(data: pd.Series, title: str = 'Time Series Analysis'):
    """
    Visualize trends, seasonality, and change points in a time series.
    
    Parameters:
    - data (pd.Series): Time series data (index = Years, values = Emissions).
    - title (str): Title prefix for plots.
    """
    # Detect period
    detected_period = detect_seasonality_period(data)
    
    # Decompose the series
    decomposition = seasonal_decompose(data, model='additive', period=detected_period, extrapolate_trend='freq')
    trend = decomposition.trend

    # Setup figure
    fig, axs = plt.subplots(3, 1, figsize=(16, 14))
    
    # 1. Original Series
    sns.lineplot(x=data.index, y=data.values, marker='o', ax=axs[0])
    axs[0].set_title(f'{title} - Original Data')
    axs[0].set_xlabel('Year')
    axs[0].set_ylabel('Emissions')
    
    # 2. Trend Component
    sns.lineplot(x=trend.index, y=trend.values, color='green', ax=axs[1])
    axs[1].set_title(f'{title} - Trend Component')
    axs[1].set_xlabel('Year')
    axs[1].set_ylabel('Trend')
    
    # 3. Change Point Detection
    algo = rpt.Pelt(model="l2").fit(data.values)
    result = algo.predict(pen=2)

    sns.lineplot(x=data.index, y=data.values, marker='o', ax=axs[2])
    for i, cp in enumerate(result[:-1]):  # Skip the last (end point)
        axs[2].axvline(x=data.index[cp], color='red', linestyle='--', label='Change Point' if i == 0 else "")
    axs[2].set_title(f'{title} - Change Points')
    axs[2].set_xlabel('Year')
    axs[2].set_ylabel('Emissions')
    axs[2].legend()

    plt.tight_layout()
    plt.show()


In [ ]:
def preprocess_emissions_data(df: pd.DataFrame, areas: list[str], items: list[int], elements: list[str]) -> pd.DataFrame:
    # Filter dataset
    df_filtered = df[
        (df["Area"].isin(areas)) &
        (df["Item Code"].isin(items)) &
        (df["Element"].isin(elements)) &
        (df["Unit"] == "kt")  # Ensure emissions are in kilotonnes
    ]
    
    # Reshape dataset to time series format
    year_columns = [col for col in df_filtered.columns if col.startswith("Y") and col[1:].isdigit()]
    df_ts = df_filtered.melt(id_vars=["Item", "Element"], value_vars=year_columns, 
                             var_name="Year", value_name="Emissions")

    # Convert Year to integer and drop NaN values
    df_ts["Year"] = df_ts["Year"].str[1:].astype(int)
    df_ts = df_ts.dropna()

    # Aggregate total emissions per year for each emission type
    df_total = df_ts.groupby(["Year", "Element"])["Emissions"].sum().unstack()

    return df_total


In [ ]:
def plot_emissions(df: pd.DataFrame, title: str):
    """
    Plots historical emissions trends.
    """
    plt.figure(figsize=(10, 5))
    for col in df.columns:
        plt.plot(df.index, df[col], marker='o', linestyle='-', label=col)
    
    plt.xlabel("Year")
    plt.ylabel("Emissions (kt)")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
def forecast_emissions(df: pd.DataFrame, forecast_years=5, model_type="ARIMA"):

    forecast_years = forecast_years + 5

    forecast_results = {"Year": list(range(df.index[-1] + 1, df.index[-1] + forecast_years + 1))}
    
    df = df.reset_index()
    df["Year"] = pd.to_datetime(df["Year"], format="%Y")
    df.set_index("Year", inplace=True)
    
    for col in df.columns:
        if model_type == "ARIMA":
            model = ARIMA(df[col], order=(3, 1, 3)).fit()
            forecast_results[col] = model.forecast(steps=forecast_years).values

        elif model_type == "SARIMA":
            model = SARIMAX(df[col], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)).fit()
            forecast_results[col] = model.forecast(steps=forecast_years).values

        elif model_type == "ETS":
            model = ExponentialSmoothing(df[col], trend="add", seasonal="add", seasonal_periods=12).fit()
            forecast_results[col] = model.forecast(forecast_years)

        elif model_type == "HMM":
            series = df[col].values.reshape(-1, 1)

            # Standardize the data
            scaler = StandardScaler()
            series_scaled = scaler.fit_transform(series)

            # Fit Gaussian HMM
            model = GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000)
            model.fit(series_scaled)

            # Start from the last observed value
            last_state = model.predict(series_scaled)[-1]
            current_state = last_state

            # Use model's means and transition probabilities to simulate future
            forecast_scaled = []
            for _ in range(forecast_years):
                next_state = np.random.choice(
                    range(model.n_components), p=model.transmat_[current_state]
                )
                next_mean = model.means_[next_state][0]
                forecast_scaled.append([next_mean])
                current_state = next_state

            # Inverse transform to get back to original scale
            forecast_values = scaler.inverse_transform(forecast_scaled).flatten()
            forecast_results[col] = forecast_values

        elif model_type == "LSTM":
            series = df[col].values.reshape(-1, 1)

            # Scale data
            scaler = MinMaxScaler()
            series_scaled = scaler.fit_transform(series)

            # Create sequences for LSTM (e.g., look back 5 years)
            look_back = 5
            X, y = [], []
            for i in range(len(series_scaled) - look_back):
                X.append(series_scaled[i:i+look_back])
                y.append(series_scaled[i+look_back])
            X, y = np.array(X), np.array(y)

            # Define LSTM model
            model = Sequential()
            model.add(LSTM(50, activation='relu', input_shape=(look_back, 1)))
            model.add(Dense(1))
            model.compile(optimizer='adam', loss='mse')

            model.fit(X, y, epochs=200, verbose=0)

            # Forecast future values
            forecast_input = series_scaled[-look_back:].reshape(1, look_back, 1)
            forecasted = []
            for _ in range(forecast_years):
                pred = model.predict(forecast_input)[0][0]
                forecasted.append(pred)
                forecast_input = np.append(forecast_input[:, 1:, :], [[[pred]]], axis=1)

            forecasted_inverse = scaler.inverse_transform(np.array(forecasted).reshape(-1, 1)).flatten()
            forecast_results[col] = forecasted_inverse

        elif model_type == "XGBoost":
            series = df[col].copy()
            
            # Create lag features
            lag = 5
            data = pd.DataFrame({f"lag_{i}": series.shift(i) for i in range(1, lag + 1)})
            data["target"] = series
            data = data.dropna()

            X = data.drop(columns="target").values
            y = data["target"].values

            # Scale features (optional but helpful for consistency)
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # Train XGBoost Regressor
            model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1)
            model.fit(X_scaled, y)

            # Forecast into the future
            last_values = series[-lag:].values.tolist()
            forecast = []
            for _ in range(forecast_years):
                input_vals = np.array(last_values[-lag:]).reshape(1, -1)
                input_scaled = scaler.transform(input_vals)
                next_pred = model.predict(input_scaled)[0]
                forecast.append(next_pred)
                last_values.append(next_pred)

            forecast_results[col] = forecast

        elif model_type == "RF":
            series = df[col].copy()

            # Create lag features
            lag = 5
            data = pd.DataFrame({f"lag_{i}": series.shift(i) for i in range(1, lag + 1)})
            data["target"] = series
            data = data.dropna()

            X = data.drop(columns="target").values
            y = data["target"].values

            # Optional: scale inputs
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # Train Random Forest
            model = RandomForestRegressor(n_estimators=200, random_state=42)
            model.fit(X_scaled, y)

            # Forecasting
            last_values = series[-lag:].values.tolist()
            forecast = []
            for _ in range(forecast_years):
                input_vals = np.array(last_values[-lag:]).reshape(1, -1)
                input_scaled = scaler.transform(input_vals)
                next_pred = model.predict(input_scaled)[0]
                forecast.append(next_pred)
                last_values.append(next_pred)

            forecast_results[col] = forecast

        else:
            raise ValueError("Unsupported model type. Choose from: ARIMA, SARIMA, ETS, XGBoost, LSTM, RF, HMM")

    return pd.DataFrame(forecast_results)

In [ ]:
def plot_forecast(df_historical: pd.DataFrame, df_forecast: pd.DataFrame, title: str) -> None:
    """
    Plots historical data with forecasted trends.
    """
    plt.figure(figsize=(10, 5))

    # Plot historical data
    for col in df_historical.columns:
        plt.plot(df_historical.index, df_historical[col], marker='o', linestyle='-', label=f"Historical {col}")
    
    # Plot forecast data
    for col in df_forecast.columns[1:]:
        plt.plot(df_forecast["Year"], df_forecast[col], marker='s', linestyle='--', label=f"Forecasted {col}")

    plt.xlabel("Year")
    plt.ylabel("Emissions (kt)")
    plt.title(title)
    plt.legend()
    plt.grid(True)

    # Turn off scientific notation
    plt.ticklabel_format(style='plain', axis='y')

    # Optional: Zoom in y-axis a little
    all_values = np.concatenate([df_historical.values.flatten(), df_forecast.iloc[:, 1:].values.flatten()])
    plt.ylim(all_values.min() * 0.95, all_values.max() * 1.05)

    plt.show()


## Task 1

### **1. Long-Term Growth (1961–2004):**
- There is a clear upward trend in N₂O emissions from 1961 through the early 2000s.
- Emissions gradually increased from around 1 kt in 1961 to over 20 kt by the early 2000s, with more rapid increases starting in the 1980s.

### **2. Sharp Increase (2004–2006):**
- Around 2004–2005, emissions experienced a very sharp spike, rising from about 21 kt to over 40 kt in a very short period.

### **3. Peak and Fluctuation (2006–2012):**
- Emissions peaked around 2008 at approximately 45 kt.
- From 2006 to 2012, emissions fluctuated significantly, staying mostly above 40 kt but showing year-to-year variability.

### **4. Sharp Drop and Recovery (Post-2012):**
- After 2012, there was a sharp drop in emissions, falling to below 25 kt.
- Since then, emissions have shown some recovery and fluctuation, with an upward trend toward 2021, reaching above 30 kt again.

### **5. No Clear Seasonal Pattern:**
- The data appears to be annual, not monthly or quarterly, so no seasonal (within-year) pattern is discernible.

In [ ]:
area = "Bangladesh"
items = [5061]  # Synthetic Fertilizers
elements = ["Emissions (N2O)"]

df_cleaned = preprocess_emissions_data(df, [area], items, elements)

df_before_2025 = df_cleaned[df_cleaned.index < 2022]

df_trimmed = df_before_2025.iloc[:-5]

plot_emissions(df_before_2025, f"Historical N2O Emissions from Synthetic Fertilizers in {area}")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=10, model_type='ETS')

plot_forecast(df_before_2025, df_forecasted, f"Forecasted N2O Emissions in {area} (Synthetic Fertilizers)")


## Task 2

### **N₂O Emissions:**
- ***From Drained Organic Soils (Item: 67291):***
    - Emissions were relatively stable at ~0.23 kt/year from 1990–1997.
    - There was a gradual increase from ~0.23 kt in 1998 to ~0.27 kt by the mid-2000s.
    - Post-2005, emissions stabilized around 0.27–0.273 kt/year through 2022.

- ***From Savanna Fires (Item: 6795):***
    - Range from ~10 kt/year to ~56 kt/year.
    - Emissions fluctuate year-to-year, with notable increases around 2010.

### **CH₄ Emissions:**
- ***From Savanna Fires (Item: 6795):***
    - From ~60 kt/year to over 620 kt/year, with major peaks around 2011 and 2021.

**Total Overall Emissions for N₂O is around 24kt and for CH₄ is around 260kt**

In [ ]:
area = "Southern Africa"
items = [67291]  # Drained Organic Soils
elements = ["Emissions (N2O)", "Emissions (CH4)"]

df_cleaned = preprocess_emissions_data(df, [area], items, elements)

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, f"Total Historical N2O & CH4 Emissions in {area}")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=5, model_type='ETS')

plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


In [ ]:
area = "Southern Africa"
items = [6795]  # Savanna Fires
elements = ["Emissions (N2O)", "Emissions (CH4)"]

df_cleaned = preprocess_emissions_data(df, [area], items, elements)

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, f"Total Historical N2O & CH4 Emissions in {area}")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=5, model_type='XGBoost')

plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


## Task 3

- Emissions efficiency was calculated using relative N₂O emissions (CO₂eq) from agricultural land due to lack of economic data.
- Forecasts predict Uganda’s emissions will increase steadily over the next decade.
- Benin is expected to maintain or slightly increase emissions over the next decade

In [ ]:
area = ["Uganda"]
item = [6995]
element = ["Emissions (CO2eq) from N2O (AR5)"]

df_uganda = preprocess_emissions_data(df, area, item, element)
df_benin = preprocess_emissions_data(df, ["Benin"], item, element)
df_uganda = df_uganda.reset_index()
df_uganda.rename(columns={'Emissions (CO2eq) from N2O (AR5)': 'Uganda'}, inplace=True)
df_benin = df_benin.reset_index()
df_benin.rename(columns={'Emissions (CO2eq) from N2O (AR5)': 'Benin'}, inplace=True)

df_eff = pd.merge(df_uganda, df_benin, on='Year')

In [ ]:
# Calculate relative efficiency
df_eff['RE_Uganda'] = df_eff['Uganda'] / df_eff['Uganda'].shift(1)
df_eff['RE_Benin'] = df_eff['Benin'] / df_eff['Benin'].shift(1)

plt.figure(figsize=(10, 5))
plt.plot(df_eff['Year'], df_eff['RE_Uganda'], marker='o', linestyle='-', label='Relative Efficiency Uganda')
plt.plot(df_eff['Year'], df_eff['RE_Benin'], marker='o', linestyle='-', label='Relative Efficiency Benin')

plt.xlabel("Year")
plt.ylabel("Relative Efficiency in %")
plt.title('Relative Efficiency')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
area = "Uganda"
item = 6995  # Agricultural Land
element = "Emissions (CO2eq) from N2O (AR5)"

df_cleaned = preprocess_emissions_data(df, [area], [item], [element])

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, "Historical CO2eq Emissions Efficiency from Agricultural Land")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=10, model_type='SARIMA')

plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


In [ ]:
area = "Benin"
item = 6995  # Agricultural Land
element = "Emissions (CO2eq) from N2O (AR5)"

df_cleaned = preprocess_emissions_data(df, [area], [item], [element])

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, "Historical CO2eq Emissions Efficiency from Agricultural Land")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=10, model_type='SARIMA')
 
plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


## Task 4

Required data for sudan is low in count and as a result the the forecast struggles to accurately forecast the emission
From the available data we can arrive that the trends are uniformly linear in both cases

In [ ]:
area = "Sudan"
item = 6818  # Waste-related
element = "Emissions (CO2eq) from CH4 (AR5)"

df_cleaned = preprocess_emissions_data(df, [area], [item], [element])

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, "Waste-Related CO2eq Emissions from CH4")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=10, model_type='ARIMA')

plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


In [ ]:
area = "Non-Annex I countries"
item = 6818  # Waste-related
element = "Emissions (CO2eq) from CH4 (AR5)"

df_cleaned = preprocess_emissions_data(df, [area], [item], [element])

df_trimmed = df_cleaned.iloc[:-5]

plot_emissions(df_cleaned, "Waste-Related CO2eq Emissions from CH4")

df_forecasted = forecast_emissions(df_trimmed, forecast_years=10, model_type='ARIMA')

plot_forecast(df_cleaned, df_forecasted, f"Forecasted Emissions in {area}")


## Task 5

The data is either empty or constant for these entries and hence no time series forecasting can be done.
since the data is empty in one and possibly wrong in the other it is not possible to forecast their emmissions for the next 10 years

In [ ]:
areas = ["Malawi"]
item = 6750  # Net Forest Conversion
element = "Emissions (CO2eq) (AR5)"

df_cleaned = preprocess_emissions_data(df, areas, [item], [element])

plot_emissions(df_cleaned, "CO2eq Emissions from Net Forest Conversion")

df_forecasted = forecast_emissions(df_cleaned, forecast_years=10, model_type='RF')

plot_forecast(df_cleaned, df_forecasted, "Forecasted CO2eq Emissions in Malawi & Afghanistan")


In [ ]:
areas = ["Afghanistan"]
item = 6750  # Net Forest Conversion
element = "Emissions (CO2eq) (AR5)"

df_cleaned = preprocess_emissions_data(df, areas, [item], [element])

plot_emissions(df_cleaned, "CO2eq Emissions from Net Forest Conversion")

df_forecasted = forecast_emissions(df_cleaned, forecast_years=10, model_type='RF')

plot_forecast(df_cleaned, df_forecasted, "Forecasted CO2eq Emissions in Malawi & Afghanistan")


## Data Analysis

In [ ]:
area = "Benin"
item = 6995  # Agricultural Land
element = "Emissions (CO2eq) from N2O (AR5)"

df_cleaned = preprocess_emissions_data(df, [area], [item], [element])

df_before_2025 = df_cleaned[df_cleaned.index < 2022]

# Example: Focus on just N2O
n2o_series = df_before_2025[element]

analyze_timeseries(n2o_series, title=f'{element} Emissions Analysis')

